In [2]:
%matplotlib inline               
import pandas as pd              
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
np.random.seed(1337)  # for reproducibility
import matplotlib.pyplot as plt

import scipy
import scipy.io.wavfile
import IPython
from sklearn.cluster import KMeans
import os 
import glob
#from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
from sklearn import metrics
from sklearn.preprocessing import StandardScaler


import random
#from keras.datasets import mnist
#from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers
from keras import  regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, Conv2D


Using TensorFlow backend.


In [3]:
#batch_size = 256
nb_classes = 8
#nb_epoch = 1

# input image dimensions
img_rows, img_cols = 40, 150
# number of convolutional filters to use
nb_filters = 8
# size of pooling area for max pooling
pool_size = (3, 3)
# convolution kernel size
kernel_size = (3, 3)

In [4]:
train_afds = pd.read_csv('/mnt/E42A-AFE5/acoustic_data_indic/train_LSTM_150_40_afds.csv')

In [5]:
test_afds = pd.read_csv('/mnt/E42A-AFE5/test_data/test_LSTM_150_40_afds.csv')

In [6]:
test_afds.groupby('lan').size()

lan
b'ben'     664
b'eng'    6888
b'hin'    3397
b'kan'     176
b'mal'      83
b'man'     280
b'mar'     102
b'ori'     185
b'tam'     776
b'tel'     756
b'urd'     112
dtype: int64

In [7]:
#test_afds = pd.read_csv('/mnt/E42A-AFE5/acoustic_data_indic/train_LSTM_25_40.csv')

In [8]:
test_afds['lan'] = test_afds['lan'].replace({"b'hindi'":"b'hin'"})

In [9]:
#Size of the total data for 8 Indian languages e.g. Bengali has 932780 examples.
train_afds.groupby('lan').size()

lan
b'ben'      279
b'eng'    12954
b'guj'      197
b'hin'     3150
b'kan'     1798
b'mal'     1537
b'mar'     1388
b'tam'      284
b'tel'     3437
dtype: int64

In [10]:
train_afds = train_afds[train_afds.lan != "b'guj'"]

In [11]:
test_afds = test_afds[test_afds.lan != "b'urd'"]
test_afds = test_afds[test_afds.lan != "b'ori'"]
test_afds = test_afds[test_afds.lan != "b'man'"]
test_afds = test_afds[test_afds.lan != "b'guz'"]

In [12]:
test_afds.head()

,0,1,2,3,4,5,6,7,8,9,...,5991,5992,5993,5994,5995,5996,5997,5998,5999,lan
0,6.081747,5.647539,5.989301,5.792731,6.299132,6.965552,7.438815,7.621581,6.987224,7.023895,...,10.177686,9.701149,9.741124,9.925967,10.120768,10.325370,10.370296,9.675091,8.603929,b'hin'
1,3.683930,7.490530,10.859599,10.329108,10.270068,10.517760,8.690377,8.769230,7.842726,8.290480,...,10.178728,9.680222,9.659084,10.271333,10.315852,10.007659,9.625746,8.983969,8.524498,b'hin'
2,5.937850,7.130089,6.449243,6.923109,7.071393,8.172868,7.501072,7.996680,9.000989,8.718126,...,16.950355,15.322862,15.939582,16.023800,16.759785,18.628219,18.746532,16.828633,14.633693,b'hin'
3,12.310539,11.145738,14.400612,12.126864,14.715648,14.669934,15.543739,15.548113,17.870400,17.453308,...,10.018422,10.042325,9.958369,9.682505,10.282967,9.799570,9.747064,8.978714,7.820707,b'hin'
4,5.591835,7.060571,6.585800,6.514170,5.082371,6.401529,7.154940,6.314321,7.414189,7.317251,...,15.417990,14.143425,13.914169,14.052438,14.712698,14.508811,13.470205,12.653405,11.836051,b'hin'


In [13]:
#Size of the total data for 8 Indian languages e.g. Bengali has 932780 examples.
test_afds.groupby('lan').size()

lan
b'ben'     664
b'eng'    6888
b'hin'    3397
b'kan'     176
b'mal'      83
b'mar'     102
b'tam'     776
b'tel'     756
dtype: int64

In [14]:
#val_afds = pd.read_csv('/home/q/Desktop/rVAD_val129_mnist_2c.csv')
#val_afds

In [15]:
X_train = train_afds.drop("lan",axis=1)
y_train = train_afds["lan"]

In [16]:
X_test = test_afds.drop("lan",axis=1)
y_test = test_afds["lan"]

In [17]:
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [18]:
y_train=le.transform(y_train)

In [19]:
y_test=le.transform(y_test)

In [20]:
y_train

array([1, 1, 1, ..., 1, 1, 1])

In [21]:
y_test

array([2, 2, 2, ..., 2, 2, 2])

le = preprocessing.LabelEncoder()
le.fit(y_val)

In [22]:
le.classes_

array(["b'ben'", "b'eng'", "b'hin'", "b'kan'", "b'mal'", "b'mar'",
       "b'tam'", "b'tel'"], dtype=object)

y_val=le.transform(y_val)

y_val

In [23]:
X_train=X_train.values
X_test=X_test.values

In [24]:
X_train.shape

(24827, 6000)

In [25]:
#Dividing the data into training and validation set
val_size = .3
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                    test_size=val_size, 
                                                    random_state=42)

#Dividing the data into validation and test set
test_size = .5
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, 
                                                    test_size=test_size, 
                                                    random_state=42)

In [26]:
le.classes_

array(["b'ben'", "b'eng'", "b'hin'", "b'kan'", "b'mal'", "b'mar'",
       "b'tam'", "b'tel'"], dtype=object)

In [27]:
y_val

array([3, 0, 1, ..., 2, 7, 1])

X_train=X_train.values
X_val=X_val.values
X_test=X_test.values

In [28]:
X_train.shape

(17378, 6000)

In [29]:
X_val.shape

(7449, 6000)

In [30]:
X_test.shape

(12842, 6000)

In [31]:
#Feature Normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_val.shape

#Feature Normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_val = scaler.transform(X_val)

In [32]:

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
#X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

input_shape = (img_rows, img_cols, 1)

In [33]:
input_shape

(40, 150, 1)

In [34]:
X_train.shape

(17378, 40, 150, 1)

In [35]:
# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)
y_val = np_utils.to_categorical(y_val, nb_classes)

In [36]:
y_val

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [37]:
y_test

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [38]:
X_train.shape

(17378, 40, 150, 1)

In [39]:
X_val.shape

(7449, 40, 150, 1)

In [40]:
X_test.shape

(12842, 40, 150, 1)

In [41]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')


X_train shape: (17378, 40, 150, 1)
17378 train samples
12842 test samples
7449 val samples


#Try running the model on a smaller dataset
X_train = X_train[0:2000]
X_test = X_test[0:2000]
X_val = X_val[0:2000]

y_train = y_train[0:2000]
y_test = y_test[0:2000]
y_val = y_val[0:2000]

In [42]:
num_classes = 8

In [48]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

#Enable gpu memory gowth
#tf.config.gpu.set_per_process_memory_growth(True)
model = Sequential()
model.add(Conv2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [54]:
model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape,W_regularizer=regularizers.l2(.1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
#model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.1))
model.add(Flatten())
#model.add(Dropout(0.1))
model.add(Dense(512,W_regularizer=regularizers.l2(.1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

/home/reverie-pc/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), input_shape=(40, 150, ..., padding="valid", kernel_regularizer=<keras.reg...)`
  after removing the cwd from sys.path.
/home/reverie-pc/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, kernel_regularizer=<keras.reg...)`
  del sys.path[0]


In [55]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 38, 148, 8)        80        
_________________________________________________________________
batch_normalization_5 (Batch (None, 38, 148, 8)        32        
_________________________________________________________________
activation_7 (Activation)    (None, 38, 148, 8)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 49, 8)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 49, 8)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4704)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               2408960   
__________

In [57]:
#Here we save our best model during training and with best accuracy on validation set
best_weights_filepath = '/mnt/acoustic_data_indic/best_weights_CNN_150_40_20_aug.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [ ]:
#Training the model
epochs=3
batch_size=32
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,verbose=2,
                    validation_data=(X_val, y_val), callbacks=[saveBestModel],shuffle=True)

In [ ]:
#reload best weights
model.load_weights(best_weights_filepath)

In [ ]:
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size,verbose = 2)
print('Test accuracy:', acc)